# 🗳️ Análisis de Posibles Votantes

Análisis de seguidores de Camilo (político) para identificar perfiles de posibles votantes.

**Fuentes:**
- `seguidoresCamilo`: listado de seguidores en Instagram
- `perfilesSeguidores`: perfiles detallados **(incluye los 16 faltantes fusionados en scraping_notebook)**

**Ajuste para gráficas:** Los perfiles con `postsCount = 0` se reemplazan por `0.1` para que tengan peso al momento de graficar (evita que desaparezcan en escalas logarítmicas o proporcionales).

---

## 1. Configuración y carga de datos

In [10]:
import os
import json
import pandas as pd
import numpy as np

# Rutas (ajustar en Colab: /content/drive/MyDrive/scraping o similar)
BASE_PATH = '/Users/geotrends/Desktop/scraping'
SCRAPERS_PATH = os.path.join(BASE_PATH, 'scrapers')

# Archivos principales: seguidores del político + perfiles detallados (con 16 faltantes ya fusionados)
SEGUIDORES_JSON = os.path.join(SCRAPERS_PATH, 'seguidoresCamilo.json')
SEGUIDORES_CSV = os.path.join(SCRAPERS_PATH, 'seguidoresCamilo.csv')  # alternativa
PERFILES_JSON = os.path.join(SCRAPERS_PATH, 'perfilesSeguidores.json')
PERFILES_CSV = os.path.join(SCRAPERS_PATH, 'perfilesSeguidores.csv')

print('📂 Configuración lista. Rutas:')
print(f'   Seguidores: {SEGUIDORES_JSON}')
print(f'   Perfiles (con 16 faltantes): {PERFILES_JSON}')

📂 Configuración lista. Rutas:
   Seguidores: /Users/geotrends/Desktop/scraping/scrapers/seguidoresCamilo.json
   Perfiles (con 16 faltantes): /Users/geotrends/Desktop/scraping/scrapers/perfilesSeguidores.json


In [11]:
# Cargar seguidores (político Camilo) - JSON o CSV
if os.path.exists(SEGUIDORES_JSON):
    with open(SEGUIDORES_JSON, 'r', encoding='utf-8') as f:
        df_seguidores = pd.DataFrame(json.load(f))
else:
    df_seguidores = pd.read_csv(SEGUIDORES_CSV, encoding='utf-8')

# Cargar perfiles detallados (con 16 faltantes ya fusionados) - JSON o CSV
if os.path.exists(PERFILES_JSON):
    with open(PERFILES_JSON, 'r', encoding='utf-8') as f:
        df_perfiles = pd.DataFrame(json.load(f))
else:
    df_perfiles = pd.read_csv(PERFILES_CSV, encoding='utf-8')

# ⚠️ Ajuste para graficar: postsCount=0 → 0.1 (para que tengan peso en gráficas, según jefe)
posts = pd.to_numeric(df_perfiles['postsCount'], errors='coerce')
ceros_posts = (posts == 0) | posts.isna()
df_perfiles['postsCount'] = posts.replace(0, 0.1).fillna(0.1)
print(f'   → {ceros_posts.sum():,} perfiles con 0 posts ajustados a 0.1 (peso en gráficas)')

print(f'\n✅ Seguidores Camilo: {len(df_seguidores):,} registros')
print(f'✅ Perfiles seguidores: {len(df_perfiles):,} registros')
print(f'\n📋 Columnas seguidores: {list(df_seguidores.columns)}')
print(f'📋 Columnas perfiles: {list(df_perfiles.columns)}')

   → 908 perfiles con 0 posts ajustados a 0.1 (peso en gráficas)

✅ Seguidores Camilo: 8,858 registros
✅ Perfiles seguidores: 8,855 registros

📋 Columnas seguidores: ['id', 'Username', 'Full Name', 'Is Verified', 'Followed By You', 'Profile URL', 'Avatar']
📋 Columnas perfiles: ['fullName', 'profilePicUrl', 'username', 'postsCount', 'followersCount', 'followsCount', 'private', 'verified', 'isBusinessAccount', 'biography']


## 2. Unificar claves y fusionar

Se unen por `username` (seguidores usa `Username`, perfiles usa `username`).

In [12]:
# Normalizar username para el merge (minúsculas)
df_seguidores['username_norm'] = df_seguidores['Username'].str.strip().str.lower()
df_perfiles['username_norm'] = df_perfiles['username'].astype(str).str.strip().str.lower()

# Quitar duplicados en perfiles (por si acaso)
df_perfiles = df_perfiles.drop_duplicates(subset=['username_norm'], keep='first')

# Merge: left join para mantener todos los seguidores
df = df_seguidores.merge(
    df_perfiles,
    on='username_norm',
    how='left',
    suffixes=('_seg', '_perf')
)

print(f'📊 Registros fusionados: {len(df):,}')
print(f'✅ Con perfil detallado: {df["followersCount"].notna().sum():,}')
print(f'⚠️ Sin perfil detallado: {df["followersCount"].isna().sum():,}')

📊 Registros fusionados: 8,858
✅ Con perfil detallado: 8,855
⚠️ Sin perfil detallado: 3


In [14]:
# Limpiar: tipos numéricos (postsCount con 0→0.1 para gráficas; followers/follows como int)
num_cols = ['postsCount', 'followersCount', 'followsCount']
for c in num_cols:
    if c in df.columns:
        s = pd.to_numeric(df[c], errors='coerce')
        if c == 'postsCount':
            df[c] = s.astype(float).where(s > 0, 0.1).fillna(0.1)  # 0 o NaN → 0.1
        else:
            df[c] = s.fillna(0).astype(int)

# Mantener columnas relevantes para el análisis
cols_keep = [
    'id', 'Username', 'Full Name', 'Is Verified', 'username_norm',
    'postsCount', 'followersCount', 'followsCount',
    'private', 'verified', 'isBusinessAccount', 'biography'
]
cols_keep = [c for c in cols_keep if c in df.columns]
df = df[cols_keep].copy()

df.head(10)

,id,Username,Full Name,Is Verified,username_norm,postsCount,followersCount,followsCount,private,verified,isBusinessAccount,biography
0,6983002406,lenilu_arte,LA ROLEÑA,False,lenilu_arte,477.0,810,3289,False,False,False,Diseño grafico ilustración fotografía🧑‍🎨\nTeni...
1,8531900302,federicogp91,Federico Gomez,False,federicogp91,19.0,719,1429,False,False,False,@gomezpiedrahita.arquitectura\n@somos.minka
2,1297342672,zuluagram,Zuluaga Gallego Andres,False,zuluagram,6.0,636,5408,False,False,False,Paid in full 💎
3,17527237421,juanmurcia_tol,Juan Murcia 🔱,False,juanmurcia_tol,0.1,311,1215,False,False,True,Asesor en:\nProyectos.\nTurísmo.\nModelo Integ...
4,2719511214,nupacris77,Cristina Santamaria,False,nupacris77,0.1,114,122,False,False,False,
5,375181604,diegoarangomelo,Diego Arango,False,diegoarangomelo,970.0,696,1225,True,False,False,Papá de Benja y Delfi\nBogotano\nPolitólogo @u...
6,3170826824,jcamilo_chaves,Juan Camilo Chaves,False,jcamilo_chaves,146.0,833,6124,False,False,False,Enjoy Art 🎨🪇/ History & Culture 🧠 /Data 💽 / Tr...
7,4730388402,mateo.r_29,Mateo Roldán,False,mateo.r_29,18.0,193,345,True,False,False,
8,58895383857,sam.inmobiliaria,SAM Inmobiliaria,False,sam.inmobiliaria,29.0,246,435,False,False,True,Asesoría de propiedad raíz honestos y personal...
9,404644875,latiendaoptica,La Tienda Optica Medellin,False,latiendaoptica,180.0,645,421,False,False,True,Tienda y Taller óptico 👓\nVenta de gafas y len...


## 3. Estadísticas descriptivas

In [23]:
num_cols = ['postsCount', 'followersCount', 'followsCount']
stats = df[num_cols].agg(['max', 'mean', 'median']).round(2)
stats.index = ['Máximo', 'Promedio', 'Mediana']
print('📈 ESTADÍSTICAS GENERALES\n')
print(stats)
print('\n---\n')
print('Cuentas privadas:', int(df['private'].sum()) if 'private' in df.columns else 'N/A')
print('Cuentas verificadas:', int(df['verified'].sum()) if 'verified' in df.columns else 'N/A')
print('Cuentas de negocio:', int(df['isBusinessAccount'].sum()) if 'isBusinessAccount' in df.columns else 'N/A')

📈 ESTADÍSTICAS GENERALES

          postsCount  followersCount  followsCount
Máximo      23042.00      1008465.00       8821.00
Promedio      251.28         1889.29       2823.88
Mediana        63.00          411.00       2208.50

---

Cuentas privadas: 4380
Cuentas verificadas: 121
Cuentas de negocio: 726


## 4. Segmentación para análisis de posibles votantes

In [16]:
# Segmentos según reach y actividad
df['segmento'] = 'regular'

# Micro-influencers (1000-10k seguidores) - alto potencial de difusión
mask_micro = (df['followersCount'] >= 1000) & (df['followersCount'] < 10000)
df.loc[mask_micro, 'segmento'] = 'micro_influencer'

# Influencers / medios (10k+ seguidores)
mask_influencer = df['followersCount'] >= 10000
df.loc[mask_influencer, 'segmento'] = 'influencer'

# Cuentas muy activas (muchos posts)
mask_activa = (df['postsCount'] >= 100) & (df['followersCount'] < 1000)
df.loc[mask_activa, 'segmento'] = 'activo'

# Cuentas casi inactivas
mask_inactivo = (df['postsCount'] <= 5) & (df['followersCount'] < 500)
df.loc[mask_inactivo, 'segmento'] = 'bajo_engagement'

# Resumen por segmento
resumen = df.groupby('segmento').agg({
    'Username': 'count',
    'followersCount': ['mean', 'sum'],
    'postsCount': 'mean',
    'private': 'sum'
}).round(1)
resumen.columns = ['cantidad', 'followers_promedio', 'followers_total', 'posts_promedio', 'cuentas_privadas']
resumen = resumen.sort_values('cantidad', ascending=False)
resumen

,cantidad,followers_promedio,followers_total,posts_promedio,cuentas_privadas
segmento,,,,,
regular,2949,363.3,1071290,36.2,1644
activo,2113,492.8,1041297,428.1,1442
micro_influencer,1903,2232.6,4248670,491.4,622
bajo_engagement,1713,122.6,210043,1.2,667
influencer,180,56466.9,10164038,1541.1,5


In [17]:
# Distribución de seguidores (percentiles)
print('📊 Percentiles de followersCount:')
for p in [50, 75, 90, 95, 99]:
    val = df['followersCount'].quantile(p/100)
    print(f'  P{p}: {val:,.0f} seguidores')

print('\n🎯 Perfiles de alto alcance (1k+ seguidores):')
alto_alcance = df[df['followersCount'] >= 1000]
print(f'   Cantidad: {len(alto_alcance):,}')
print(f'   Alcance total: {alto_alcance["followersCount"].sum():,.0f}')

📊 Percentiles de followersCount:
  P50: 411 seguidores
  P75: 949 seguidores
  P90: 1,912 seguidores
  P95: 3,714 seguidores
  P99: 21,394 seguidores

🎯 Perfiles de alto alcance (1k+ seguidores):
   Cantidad: 2,083
   Alcance total: 14,412,708


### Gráfico de dispersión interactivo: Seguidores vs Seguidos

- **Eje Y:** Followers (seguidores) en escala logarítmica
- **Eje X:** Following (seguidos) en escala logarítmica  
- **Tamaño de burbujas:** Cantidad de posts (postsCount)
- **Interactivo:** Pasa el mouse sobre cada punto para ver el perfil (nombre, usuario, seguidores, posts). Zoom, pan y doble clic para resetear vista.

In [22]:
# Gráfico interactivo con Plotly (hover/popups, zoom, pan)
import plotly.express as px

# Datos: evitar 0 para ejes log
plot_df = df[df['followersCount'].notna() & df['followsCount'].notna()].copy()
plot_df['followers_plot'] = np.maximum(plot_df['followersCount'], 1)
plot_df['follows_plot'] = np.maximum(plot_df['followsCount'], 1)

fig = px.scatter(
    plot_df,
    x='follows_plot',
    y='followers_plot',
    size='postsCount',
    color='segmento' if 'segmento' in plot_df.columns else None,
    hover_name='Full Name',
    hover_data={
        'Username': True,
        'followersCount': True,
        'followsCount': True,
        'postsCount': True,
        'segmento': True,
        'followers_plot': False,
        'follows_plot': False,
    },
    log_x=True,
    log_y=True,
    size_max=50,
    title='Distribución de perfiles: Seguidores vs Seguidos (tamaño = posts)',
    labels={
        'follows_plot': 'Following (seguidos)',
        'followers_plot': 'Followers (seguidores)',
        'followersCount': 'Seguidores',
        'followsCount': 'Seguidos',
        'postsCount': 'Posts',
        'segmento': 'Segmento',
    },
)
fig.update_traces(
    marker=dict(line=dict(width=0.5, color='white')),
    selector=dict(mode='markers')
)
fig.update_layout(
    height=700,
    template='plotly_white',
    legend=dict(title='Segmento', yanchor='top', y=1, xanchor='left', x=1.02),
    xaxis_title='Following (seguidos) - escala log',
    yaxis_title='Followers (seguidores) - escala log',
)
fig.show()

### Gráfico filtrado: >500 seguidos, 200–5000 seguidores

Misma distribución pero solo perfiles que siguen a **más de 500 personas** y tienen **entre 200 y 5000 seguidores**.

In [26]:
# Mismo gráfico, filtrado: >500 seguidos Y 200-5000 seguidores
plot_df_filt = plot_df[
    (plot_df['followsCount'] > 500) & 
    (plot_df['followersCount'] >= 200) & 
    (plot_df['followersCount'] <= 5000)
].copy()

fig2 = px.scatter(
    plot_df_filt,
    x='follows_plot',
    y='followers_plot',
    size='postsCount',
    color='segmento' if 'segmento' in plot_df_filt.columns else None,
    hover_name='Full Name',
    hover_data={
        'Username': True,
        'followersCount': True,
        'followsCount': True,
        'postsCount': True,
        'segmento': True,
        'followers_plot': False,
        'follows_plot': False,
    },
    log_x=True,
    log_y=True,
    size_max=50,
    title='Filtrado: >500 seguidos, 200-5000 seguidores (tamaño = posts)',
    labels={
        'follows_plot': 'Following (seguidos)',
        'followers_plot': 'Followers (seguidores)',
        'followersCount': 'Seguidores',
        'followsCount': 'Seguidos',
        'postsCount': 'Posts',
        'segmento': 'Segmento',
    },
)
fig2.update_traces(
    marker=dict(line=dict(width=0.5, color='white')),
    selector=dict(mode='markers')
)
fig2.update_layout(
    height=700,
    template='plotly_white',
    legend=dict(title='Segmento', yanchor='top', y=1, xanchor='left', x=1.02),
    xaxis_title='Following (seguidos) - escala log',
    yaxis_title='Followers (seguidores) - escala log',
)
fig2.show()

print(f'📊 Perfiles en el filtro: {len(plot_df_filt):,} de {len(plot_df):,} totales')

📊 Perfiles en el filtro: 5,696 de 8,858 totales


## 5. Identificación de localización (Medellín / Antioquia)

Búsqueda en biografía para perfiles que mencionan la región.

In [18]:
df['biografia_lower'] = df['biography'].fillna('').astype(str).str.lower()

keywords_local = ['medellín', 'medellin', 'antioquia', 'envigado', 'sabaneta', 'bello', 'iturbe']
df['es_local'] = df['biografia_lower'].apply(
    lambda x: any(k in x for k in keywords_local)
)

print('📍 Perfiles con indicio de localización (Medellín/Antioquia):')
print(f'   Cantidad: {df["es_local"].sum():,}')
print(f'   % del total: {100*df["es_local"].mean():.1f}%')

df_local = df[df['es_local']]
if len(df_local) > 0:
    print('\nEjemplos:')
    display(df_local[['Username', 'Full Name', 'followersCount', 'biography']].head(5))

📍 Perfiles con indicio de localización (Medellín/Antioquia):
   Cantidad: 387
   % del total: 4.4%

Ejemplos:


,Username,Full Name,followersCount,biography
8,sam.inmobiliaria,SAM Inmobiliaria,246,Asesoría de propiedad raíz honestos y personal...
12,moni.montespe,Moni A. Montes,647,📍Medellín - Colombia\n🔅Orgullosamente Mamá de ...
27,greeneventsantioquia,Greenevents Antioquia,225,-Experiencias-Teambuilding-Activides sostenibl...
30,tallerpruebadestado,Tallerpruebadestado,2032,Prueba de Estado está localizado en Envigado ...
41,anabenavidesd,Ana Maria Benavides,708,"Botánica, protegiendo las plantas en peligro d..."


## 6. Top perfiles por alcance (posibles multiplicadores)

In [19]:
top = df.nlargest(20, 'followersCount')[['Username', 'Full Name', 'followersCount', 'postsCount', 'verified', 'segmento', 'es_local']]
top

,Username,Full Name,followersCount,postsCount,verified,segmento,es_local
1089,matador000,matador,1008465,7362.0,True,influencer,False
6649,claudiaelena,Claudia Elena Vásquez,933942,2418.0,True,influencer,False
1133,en_el_ruedo_,En El Ruedo,511224,8355.0,True,influencer,False
7791,alexarodriguezzz,"ALEXA | Fitness, nutrición y entrenamiento",484400,1287.0,True,influencer,False
235,facial_fitness,Juli Bravo | Yoga Facial | Masaje Japonés | Ac...,250750,384.0,True,influencer,True
7329,suculento.co,Suculento.co,234624,909.0,False,influencer,True
6149,nidis_agamez_,Nidis Agamex,220210,5319.0,False,influencer,False
1612,ursulasuarezoficial_1,Úrsula Suárez,176056,1606.0,True,influencer,True
1291,nibarguen,Nicolas Ibarguen,164063,426.0,True,influencer,False
4732,angelicalozano10,Angélica Lozano | SENADO #VERDE10 💚🌻,152274,2843.0,True,influencer,False


In [ ]:
# Exportar dataset fusionado para uso posterior
OUTPUT_CSV = os.path.join(SCRAPERS_PATH, 'seguidores_con_perfiles.csv')
df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')
print(f'💾 Guardado: {OUTPUT_CSV}')
print(f'   {len(df):,} filas')